<a href="https://colab.research.google.com/github/poongodimsa/DataScience-projects-for-vidiq/blob/main/CMAudit_SegregateCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segregate CM data

###Specifications

####  Step 01:

Add CM monthly and Yearly data together but to differentiate data, add a separate column name "Plan Type" and label monthly data as "Monthly" and Yearly data as "Annual" 


#### Step 2:

Now we need to separate/sort out data by different platform data using "Charge ID". Create separate sheets/Tab to move below data 

o	Transactions starting with ch_& re_ or py_ -----> Stripe

o	Transactions Starting with number -------> Paypal  ( Or in the middle separate by -Com)

o	Transactions starting with tr_ -----> Apple & Google


### Script
---

#### Imports,prepartions and functions

In [ ]:
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from datetime import date
from dateutil.relativedelta import *
import shutil
import xlsxwriter


###### Preparations
---


In [ ]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/"

inpathCM = path + "CM Audit/"

outpath = path + "Results/CM Segregated/"
archpath = path + "Results/Archives/"

#Create Results folder if missing
if not os.path.isdir(outpath):
  os.mkdir(outpath)

#Create archive folder if missing
if not os.path.isdir(archpath):
  os.mkdir(archpath)


Mounted at /content/gdrive


#### Segregate ChartMogul

In [ ]:
for item in sorted(os.listdir(inpathCM)):
  
  print(item)
  if os.path.isdir(inpathCM  + item):
    ###### Combine Monthly and yearly data differentiating using 'Plan Type' column ############
    for file in os.listdir(inpathCM + item ):
      #print (file)
      
      if "Mon" in file:     #Monthly data
        dfmon = pd.read_csv(inpathCM + item + "/" +  file , na_filter=False, index_col=False)
      else:
        dfyear = pd.read_csv(inpathCM + item + "/" +  file , na_filter=False, index_col=False)

    colcnt = dfmon.shape[1]
    dfmon.insert(colcnt,"Plan Type", "Monthly")
    dfyear.insert(colcnt,"Plan Type", "Yearly" )
    
    #Concat the datasets
    dfcombine = pd.concat([dfmon,dfyear])
   
    ###### Segregate based on 'Charge Id' ##########
    #Transactions starting with ch_& re_ or py_ -----> Stripe
    dfs = dfcombine[dfcombine['Charge ID'].str.contains("ch_|re_|py_") ]
    #Transactions Starting with number -------> Paypal ( Or in the middle separate by -Com)
    dfpp = dfcombine[dfcombine['Charge ID'].str.contains("-Com") ]
    #Transactions starting with tr_ -----> Apple & Google
    dfag = dfcombine[dfcombine['Charge ID'].str.contains("tr_") ]
    #Others
    dfmerge=pd.concat([dfs,dfag,dfpp])
    dfother = dfcombine[~dfcombine["Charge ID"].isin(dfmerge["Charge ID"])]


    filename = "CM_" + item[2:] + ".xlsx"

    writer = pd.ExcelWriter(outpath + filename, engine='xlsxwriter')
    dfs.to_excel(writer,sheet_name="Stripe", index=False)
    dfpp.to_excel(writer,sheet_name="Paypal", index=False)
    dfag.to_excel(writer,sheet_name="Apple&Google", index=False)
    dfother.to_excel(writer,sheet_name="Others", index=False)

    writer.save()
    writer.close()

print("Done!")

2022.06


/usr/local/lib/python3.7/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


2022.07
Done!
